In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import time

In [101]:
df = pd.read_csv('df_for_model.csv')
df = df.drop('Unnamed: 0',axis=1)

In [102]:
SiteEnergyUse_discrete = pd.qcut(df['SiteEnergyUse(kBtu)'],3, labels = False)
GHGEmission_discrete = pd.qcut(df['GHGEmissions(MetricTonsCO2e)'],3, labels = False)
df.insert(11, 'SiteEnergyUse_discrete', SiteEnergyUse_discrete)
df.insert(12, 'GHGEmission_discrete', GHGEmission_discrete)

In [103]:
df['SiteEnergyUse(kBtu)_log'] = np.log1p(df['SiteEnergyUse(kBtu)'])
df['GHGEmissions(MetricTonsCO2e)_log'] = np.log1p(df['GHGEmissions(MetricTonsCO2e)'])

In [104]:
df=df.drop(df[df['PrimaryPropertyType']=='Hospital'].index)

In [105]:
df.columns

Index(['PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
       'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'SiteEnergyUse_discrete',
       'GHGEmission_discrete', 'GHGEmissions(MetricTonsCO2e)',
       'MedianEnergyUse', 'MedianGHGEmission', 'SiteEnergyUse(kBtu)_log',
       'GHGEmissions(MetricTonsCO2e)_log'],
      dtype='object')

In [106]:
data=df['Neighborhood'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
neighborhood_enc = enc.fit_transform(data).toarray()
neighborhood_enc.shape

(1800, 13)

In [107]:
data=df['PrimaryPropertyType'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
PropertyType_enc = enc.fit_transform(data).toarray()
PropertyType_enc.shape

(1800, 24)

In [108]:
X_cat = np.concatenate((neighborhood_enc,PropertyType_enc),axis=1)
X_cat.shape

(1800, 37)

In [109]:
#X_cat = np.concatenate((X_cat,df['SiteEnergyUse_discrete'].to_numpy().reshape(-1,1)),axis=1)

In [110]:
X_quant = df[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)']]
X_quant.values.shape

(1800, 4)

In [111]:
Y_energy_log = df['SiteEnergyUse(kBtu)_log']
Y_ghg_log = df['GHGEmissions(MetricTonsCO2e)_log']
Y_energy_bis = df['SiteEnergyUse(kBtu)']
Y_ghg_bis = df['GHGEmissions(MetricTonsCO2e)']

In [112]:
X_cat_train, X_cat_test, X_quant_train, X_quant_test, Y_energy_log_train, Y_energy_log_test, Y_ghg_log_train, Y_ghg_log_test, Y_energy_bis_train, Y_energy_bis_test, Y_ghg_bis_train, Y_ghg_bis_test = train_test_split(X_cat, X_quant, Y_energy_log, Y_ghg_log, Y_energy_bis, Y_ghg_bis, test_size=0.3, stratify = df['PrimaryPropertyType'], random_state = 0)

In [113]:
std = preprocessing.StandardScaler()
X_quant_train_scaled = std.fit_transform(X_quant_train)
X_quant_test_scaled = std.transform(X_quant_test)

In [114]:
X_train = np.concatenate((X_cat_train,X_quant_train_scaled),axis=1)
X_test = np.concatenate((X_cat_test,X_quant_test_scaled),axis=1)

In [115]:
# Régression linéaire sur cibles en log
lr_energy = linear_model.LinearRegression()
lr_energy.fit(X_train,Y_energy_log_train)

print('R² test sur variable log:', round(metrics.r2_score(Y_energy_log_test,lr_energy.predict(X_test)),3))
print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))

print('R² train sur variable log:',round(metrics.r2_score(Y_energy_log_train,lr_energy.predict(X_train)),3))
print('R² train sur variable orig:',round(metrics.r2_score(np.expm1(Y_energy_log_train),np.expm1(lr_energy.predict(X_train))),3))

R² test sur variable log: 0.459
R² test sur variable origine: -2.167
R² train sur variable log: 0.532
R² train sur variable orig: -46.834


In [116]:
# Régression linéaire sur cibles non transformées
lr_energy_bis = linear_model.LinearRegression()
lr_energy_bis.fit(X_train,Y_energy_bis_train)

print('R² test :', round(metrics.r2_score(Y_energy_bis_test,lr_energy_bis.predict(X_test)),3))
print('R² train :',round(metrics.r2_score(Y_energy_bis_train,lr_energy_bis.predict(X_train)),3))

R² test : 0.57
R² train : 0.744


In [117]:
recap_energy = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy')
recap_ghg = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG')

In [118]:
recap_energy_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy log')
recap_ghg_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG log')

In [119]:
def perf(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(Y_test,model_gs.predict(X_test)),5)     
    RMSE = round(metrics.mean_squared_error(Y_test,model_gs.predict(X_test), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score, RMSE, pred_time, best_param



def recap_table(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [120]:
def perf_log(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)     
    RMSE = round(metrics.mean_squared_error(np.expm1(Y_test),np.expm1(model_gs.predict(X_test)), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score,RMSE, pred_time, best_param



def recap_table_log(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [121]:
LinearReg = linear_model.LinearRegression();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,LinearReg,{})

In [122]:
ridge = Ridge();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [123]:
lasso = Lasso();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.5667167905275192e+16, tolerance: 97632493840804.52
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.700990073844917e+16, tolerance: 117389717589443.67
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.680987103579774e+16, tolerance: 117378681142148.64
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py

In [124]:
svr = SVR(kernel = 'linear');
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [125]:
svr = SVR();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [126]:
rfr = RandomForestRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [127]:
gb = LGBMRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, gb, {'n_estimators': [10, 50, 100, 150, 200],'max_depth':range(5,16, 2), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1,1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart'], 'num_leaves':[30]})

In [128]:
recap_energy

,R²,RMSE,Prediction time,Best parameters
Model for energy,,,,
LinearRegression(),0.57033,8.50073e+06,0.002003,{}
Ridge(),0.59516,8.25143e+06,0.008001,{'alpha': 14.174742}
Lasso(),0.57121,8.49202e+06,0.002994,{'alpha': 100.0}
SVR(kernel='linear'),-0.11193,1.3675e+07,0.055482,"{'C': 4, 'epsilon': 0.1}"
SVR(),-0.11141,1.36718e+07,0.081997,"{'C': 4, 'epsilon': 0.1, 'kernel': 'poly'}"
RandomForestRegressor(),0.38555,1.01655e+07,0.038005,"{'max_depth': 7, 'min_samples_leaf': 2, 'min_s..."
LGBMRegressor(),0.56888,8.51505e+06,0.003992,"{'boosting_type': 'dart', 'learning_rate': 0.1..."


In [129]:
recap_ghg

,R²,RMSE,Prediction time,Best parameters
Model for GHG,,,,
LinearRegression(),0.22296,306.559,0.003004,{}
Ridge(),0.31231,288.396,0.002,{'alpha': 86.9749}
Lasso(),0.2722,296.687,0.001001,{'alpha': 8.111308}
SVR(kernel='linear'),0.27067,297,0.058002,"{'C': 4, 'epsilon': 0.3}"
SVR(),0.07312,334.815,0.088006,"{'C': 4, 'epsilon': 0.7, 'kernel': 'sigmoid'}"
RandomForestRegressor(),0.14709,321.178,0.030545,"{'max_depth': 6, 'min_samples_leaf': 2, 'min_s..."
LGBMRegressor(),0.43691,260.965,0.009,"{'boosting_type': 'goss', 'learning_rate': 0.0..."


In [130]:
LinearReg = linear_model.LinearRegression();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,LinearReg,{})

In [131]:
ridge = Ridge();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [132]:
lasso = Lasso();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.9886457526739605, tolerance: 0.17820496983953157
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.018911797805913, tolerance: 0.17820496983953157
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.662419147172727, tolerance: 0.17820496983953157
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: Co

In [133]:
svr = SVR(kernel = 'linear');
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [134]:
svr = SVR();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [135]:
rfr = RandomForestRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [136]:
gb = LGBMRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, gb, {'n_estimators': [10, 50, 100, 150, 200],'max_depth':range(5,10), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1,1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart'], 'num_leaves':[30]})

In [137]:
recap_energy_log

,R²,RMSE,Prediction time,Best parameters
Model for energy log,,,,
LinearRegression(),-2.16701,2.30788e+07,0.003001,{}
Ridge(),-0.09341,1.35606e+07,0.001998,{'alpha': 100.0}
Lasso(),-2.15825,2.30469e+07,0.002,{'alpha': 0.001072}
SVR(kernel='linear'),-2.65891,2.48065e+07,0.019999,"{'C': 1, 'epsilon': 0.9}"
SVR(),0.6535,7.63374e+06,0.081006,"{'C': 2, 'epsilon': 0.1, 'kernel': 'rbf'}"
RandomForestRegressor(),0.56763,8.52741e+06,0.184179,"{'max_depth': 14, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.61342,8.06327e+06,0.008998,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [138]:
recap_ghg_log

,R²,RMSE,Prediction time,Best parameters
Model for GHG log,,,,
LinearRegression(),-1.0849,502.153,0.002,{}
Ridge(),0.03982,340.776,0.002,{'alpha': 100.0}
Lasso(),-0.9864,490.147,0.004003,{'alpha': 0.001072}
SVR(kernel='linear'),-8.47547,1070.52,0.037002,"{'C': 4, 'epsilon': 0.7}"
SVR(),0.48211,250.272,0.034003,"{'C': 2, 'epsilon': 0.5, 'kernel': 'rbf'}"
RandomForestRegressor(),0.40524,268.203,0.047005,"{'max_depth': 15, 'min_samples_leaf': 2, 'min_..."
LGBMRegressor(),0.43902,260.476,0.009001,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [139]:
recap_energy_log.iloc[-1,-1]

{'boosting_type': 'gbdt',
 'learning_rate': 0.05,
 'max_depth': 5,
 'n_estimators': 100,
 'num_leaves': 30}